In [1]:
import numpy as np, matplotlib.pyplot as plt, seaborn as sns
import pandas as pd, scipy.stats as stats
from tqdm import tqdm
import torch, torch.nn as nn, torch.optim as optim, torch.nn.functional as F
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Neural Network Strategy Set Up
This code will set up a feed forward neural network as a base model to test the algorithm.

In [3]:
class Strategy(nn.Module):
    def __init__(self, hiddenNodes, numTradingDates):
        super(Strategy, self).__init__()
        
        # Price of the Options
        self.price = nn.Parameter(torch.tensor(0, dtype = torch.float32))
        
        # Trading/Hedging Strategy
        self.steps = nn.ModuleList()
        for i in range(numTradingDates):
            h_i = nn.Sequential(nn.Linear(1, hiddenNodes)
                               , nn.ReLu()
                               , nn.Linear(hiddenNodes, 1)
                               )
            
            self.steps.append(h_i)
        
    def forward(self, x): # x is a subset of paths of price process
        dS = torch.diff(x)
        ret = torch.zeros_like(dS[:,0]) + self.price
        
        for i in range(len(self.steps)):
            logS = torch.log(x[i,:])
            h_i = self.steps[i]
            H_i = h_i(logS.unsqueeze(-1)) # separates logS (single list) values into list of single-valued lists.
            ret += dS[:, i] * H_i.flatten()
            return ret
        